# Imports

In [ ]:
#for some stupid reason, plt crashes unless I do this

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(np.zeros((2,3)))

In [ ]:
import warnings
import math
import random
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image
import pickle
import gzip

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
 

from tqdm import tqdm

from torch_geometric.data import Data, Batch, DataLoader
from torch_geometric.utils import*
from torch_geometric.nn import GCNConv, CGConv, GINConv, Sequential, global_mean_pool, global_add_pool
from scipy.sparse.csgraph import csgraph_from_dense, csgraph_to_dense
from scipy.sparse import*

import json
import pdb
import scipy.sparse
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import copy
import numpy as np
import networkx as nx
import time
import os

import AllClassicalAlgos
from AllClassicalAlgos import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings("ignore")


In [ ]:
torch.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(torch.version.cuda)

# Helper Funcs

In [ ]:
from torch_geometric.utils import to_scipy_sparse_matrix, from_scipy_sparse_matrix
from scipy.sparse import csr_matrix

def pyg_to_csr(pyg_data):
    edge_index = pyg_data.edge_index.numpy()
    num_nodes = pyg_data.num_nodes

    # Combine forward and reverse edges
    edges = np.hstack([edge_index, np.flip(edge_index, axis=0)])

    # Create a binary adjacency matrix in COO format, then convert to CSR
    adj_coo = coo_matrix((np.ones(edges.shape[1]), edges), shape=(num_nodes, num_nodes))
    adj_csr = adj_coo.tocsr()

    return adj_csr

def csr_to_pyg(csr_matrix, features, label):
    # Converts a Scipy CSR matrix to a PyG Data object.
    edge_index, _ = from_scipy_sparse_matrix(csr_matrix)
    return Data(x=features, edge_index=edge_index, y=label)

In [ ]:
def remove_redundant_edges_csr(adj_csr):
    # Only keep the upper triangular part to remove redundant edges
    adj_single = triu(adj_csr, format='csr')
    return adj_single

In [ ]:
#   new version to handle edge cases of isolated nodes
'''does not work'''
def data_to_networkx(data):
    '''this might not work'''
    G = nx.Graph()
    num_nodes = data.x.shape[0]
    G.add_nodes_from(range(num_nodes))  # This line adds all nodes to the graph
    edge_index = data.edge_index.numpy()
    G.add_edges_from(edge_index.T)
    return G

# Load GNN-RE Dataset

In [ ]:
re_adj_full = np.load(os.path.join('GNN-RE-DS','adj_full.npz')) 
re_adj_train = np.load(os.path.join('GNN-RE-DS','adj_train.npz')) 
re_cell = np.load(os.path.join('GNN-RE-DS','cell.npy')) 
re_feats = np.load(os.path.join('GNN-RE-DS','feats.npy')) 

In [ ]:
def gnn_re_load_data(prefix='GNN-RE-DS', normalize=True):
    """
    FROM GNN-RE REPO:
    Load the various data files residing in the `prefix` directory.
    Files to be loaded:
        adj_full.npz        sparse matrix in CSR format, stored as scipy.sparse.csr_matrix
                            The shape is N by N. Non-zeros in the matrix correspond to all
                            the edges in the full graph. It doesn't matter if the two nodes
                            connected by an edge are training, validation or test nodes.
                            For unweighted graph, the non-zeros are all 1.
        adj_train.npz       sparse matrix in CSR format, stored as a scipy.sparse.csr_matrix
                            The shape is also N by N. However, non-zeros in the matrix only
                            correspond to edges connecting two training nodes. The graph
                            sampler only picks nodes/edges from this adj_train, not adj_full.
                            Therefore, neither the attribute information nor the structural
                            information are revealed during training. Also, note that only
                            a x N rows and cols of adj_train contains non-zeros. For
                            unweighted graph, the non-zeros are all 1.
        role.json           a dict of three keys. Key 'tr' corresponds to the list of all
                              'tr':     list of all training node indices
                              'va':     list of all validation node indices
                              'te':     list of all test node indices
                            Note that in the raw data, nodes may have string-type ID. You
                            need to re-assign numerical ID (0 to N-1) to the nodes, so that
                            you can index into the matrices of adj, features and class labels.
        class_map.json      a dict of length N. Each key is a node index, and each value is
                            either a length C binary list (for multi-class classification)
                            or an integer scalar (0 to C-1, for single-class classification).
        feats.npz           a numpy array of shape N by F. Row i corresponds to the attribute
                            vector of node i.

    Inputs:
        prefix              string, directory containing the above graph related files
        normalize           bool, whether or not to normalize the node features

    Outputs:
        adj_full            scipy sparse CSR (shape N x N, |E| non-zeros), the adj matrix of
                            the full graph, with N being total num of train + val + test nodes.
        adj_train           scipy sparse CSR (shape N x N, |E'| non-zeros), the adj matrix of
                            the training graph. While the shape is the same as adj_full, the
                            rows/cols corresponding to val/test nodes in adj_train are all-zero.
        feats               np array (shape N x f), the node feature matrix, with f being the
                            length of each node feature vector.
        class_map           dict, where key is the node ID and value is the classes this node
                            belongs to.
        role                dict, where keys are: 'tr' for train, 'va' for validation and 'te'
                            for test nodes. The value is the list of IDs of nodes belonging to
                            the train/val/test sets.
    """
    adj_full = scipy.sparse.load_npz('./{}/adj_full.npz'.format(prefix)).astype(np.bool)
    adj_train = scipy.sparse.load_npz('./{}/adj_train.npz'.format(prefix)).astype(np.bool)
    role = json.load(open('./{}/role.json'.format(prefix)))
    feats = np.load('./{}/feats.npy'.format(prefix))
    class_map = json.load(open('./{}/class_map.json'.format(prefix)))
    class_map = {int(k):v for k,v in class_map.items()}
    assert len(class_map) == feats.shape[0]
    # ---- normalize feats ----
    train_nodes = np.array(list(set(adj_train.nonzero()[0])))
    train_feats = feats[train_nodes]
    scaler = StandardScaler()
    scaler.fit(train_feats)
    feats = scaler.transform(feats)
    # -------------------------
    return adj_full, adj_train, feats, class_map, role

In [ ]:
re_adj_full, re_adj_train, re_feats, re_class_map, re_role = gnn_re_load_data()

In [ ]:
re_role.keys()

In [ ]:
re_adj_full

In [ ]:
re_adj_train

In [ ]:
re_feats.shape

In [ ]:
len(re_class_map.keys())

In [ ]:
num_nodes = re_feats.shape[0]

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[torch.tensor(re_role['tr'])] = True

val_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask[torch.tensor(re_role['va'])] = True

test_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask[torch.tensor(re_role['te'])] = True

# train_mask = re_role['tr']
# val_mask = re_role['va']
# test_mask = re_role['te']
row, col = re_adj_full.nonzero()
edge_index = torch.tensor([row, col], dtype=torch.long)
# Get the keys in sorted order
keys = sorted(re_class_map.keys())

# Create a numpy array with the class values in the same order
re_class_map_np = np.array([re_class_map[key] for key in keys])

# Now you can create your PyG Data object
gnn_re = Data(edge_index=edge_index, x=torch.from_numpy(re_feats), y=torch.from_numpy(re_class_map_np).long())


In [ ]:
type(gnn_re)

# Remove self-loops from GNN-RE

In [ ]:
is_undirected(gnn_re.edge_index)

In [ ]:
contains_self_loops(gnn_re.edge_index)

In [ ]:
newE, newA = remove_self_loops(gnn_re.edge_index, gnn_re.edge_attr)

In [ ]:
gnn_re.edge_index = newE
gnn_re.edge_attr = newA

In [ ]:
contains_self_loops(gnn_re.edge_index)

In [ ]:
print(gnn_re) #the graph is not connected. 

In [ ]:
numREFeatures = 34

In [ ]:
def check_symmetry(graphs):
    """
    Checks whether each graph in a list of graphs (or a single graph) is symmetric.
    If a graph is not symmetric, it prints the index of that graph.

    Parameters
    ----------
    graphs : torch_geometric.data.Data or list of torch_geometric.data.Data
        The graph or list of graphs to check.

    Returns
    -------
    None
    """
    def is_symmetric(matrix):
        """Checks whether a matrix is symmetric."""
        return torch.all(matrix == matrix.T)

    # Handle the case where graphs is not a list
    if not isinstance(graphs, list):
        graphs = [graphs]

    sym = True
    for idx, graph in enumerate(graphs):
        # Convert the edge_index tensor to an adjacency matrix
        adj_matrix = torch.zeros((graph.num_nodes, graph.num_nodes))
        edge_index = graph.edge_index.t().tolist()
        for edge in edge_index:
            adj_matrix[edge[0], edge[1]] = 1
            adj_matrix[edge[1], edge[0]] = 1
        
        if not is_symmetric(adj_matrix):
            sym = False
            print(f"Graph at index {idx} is not symmetric.")
    return sym

In [ ]:
# check_symmetry(gnn_re)

# Get individual Graphs from GNN-RE

In [ ]:
def count_individual_graphs(graph):
    nx_graph = nx.Graph()
    nx_graph.add_edges_from(graph.edge_index.T.tolist())

    # Compute the connected components
    components = nx.connected_components(nx_graph)

    # Count the number of individual graphs
    num_graphs = len(list(components))

    return num_graphs

In [ ]:
count_individual_graphs(gnn_re) #there are 43 disconnected small graphs which make up this large graph 

In [ ]:
def get_individual_graphs(graph):
    # Create a NetworkX graph from the input graph
    nx_graph = nx.Graph()
    nx_graph.add_edges_from(graph.edge_index.T.tolist())

    # Compute the connected components
    components = list(nx.connected_components(nx_graph))

    # Create PyTorch Geometric graphs for each component
    individual_graphs = []
    for component in tqdm(components):
        # Create a mapping from the original node index to the new node index in the subgraph
        index_mapping = {original_index: new_index for new_index, original_index in enumerate(component)}
        
        # Convert the component to a list of nodes
        component_nodes = list(component)
        
        # Extract the subgraph for the current component
        subgraph = nx_graph.subgraph(component)
        
        # Remap the edge indices to match the new node indices in the subgraph
        remapped_edges = [[index_mapping[edge[0]], index_mapping[edge[1]]] for edge in subgraph.edges()]
        component_edges = torch.tensor(remapped_edges).T

        # Extract node features for the current component
        component_feats = graph.x[torch.tensor(component_nodes)]
        
        # Create a PyTorch Geometric Data object for the subgraph
        individual_graphs.append(Data(x=component_feats, edge_index=component_edges))

    return individual_graphs


In [ ]:
gnn_re_43 = get_individual_graphs(gnn_re)

In [ ]:
gnn_re_43 = sorted(gnn_re_43, key=lambda graph: graph.num_nodes)

In [ ]:
gnn_re_43

In [ ]:
check_symmetry(gnn_re_43)

In [ ]:
def compute_graph_diameter(graph):
    '''computes the diameter of a PYG graph. this function runs in polynomial time but can still take a long time to run'''
    # Create a NetworkX graph from the input graph

    nx_graph = nx.Graph()
    nx_graph.add_edges_from(graph.edge_index.T.tolist())

    # Compute the diameter using NetworkX
    diameter = nx.diameter(nx_graph)

    return diameter

In [ ]:
#Takes 14 min to run
# diameters = []
# j = 0 
# for subG in tqdm(gnn_re_43):
#     diameters.append([j, subG, compute_graph_diameter(subG)])
#     j += 1


In [ ]:
#each graph and it's diameter


diameters = [[0, Data(x=[3, 34], edge_index=[2, 2]), 2],
 [1, Data(x=[3, 34], edge_index=[2, 2]), 2],
 [2, Data(x=[6, 34], edge_index=[2, 5]), 5],
 [3, Data(x=[6, 34], edge_index=[2, 5]), 5],
 [4, Data(x=[10, 34], edge_index=[2, 9]), 9],
 [5, Data(x=[12, 34], edge_index=[2, 19]), 4],
 [6, Data(x=[13, 34], edge_index=[2, 13]), 6],
 [7, Data(x=[16, 34], edge_index=[2, 20]), 5],
 [8, Data(x=[35, 34], edge_index=[2, 53]), 7],
 [9, Data(x=[44, 34], edge_index=[2, 71]), 8],
 [10, Data(x=[61, 34], edge_index=[2, 109]), 7],
 [11, Data(x=[71, 34], edge_index=[2, 90]), 20],
 [12, Data(x=[79, 34], edge_index=[2, 133]), 10],
 [13, Data(x=[87, 34], edge_index=[2, 150]), 10],
 [14, Data(x=[119, 34], edge_index=[2, 208]), 11],
 [15, Data(x=[131, 34], edge_index=[2, 231]), 11],
 [16, Data(x=[149, 34], edge_index=[2, 242]), 10],
 [17, Data(x=[207, 34], edge_index=[2, 416]), 11],
 [18, Data(x=[222, 34], edge_index=[2, 317]), 22],
 [19, Data(x=[239, 34], edge_index=[2, 461]), 14],
 [20, Data(x=[255, 34], edge_index=[2, 494]), 15],
 [21, Data(x=[309, 34], edge_index=[2, 591]), 12],
 [22, Data(x=[342, 34], edge_index=[2, 649]), 13],
 [23, Data(x=[484, 34], edge_index=[2, 718]), 29],
 [24, Data(x=[576, 34], edge_index=[2, 1065]), 13],
 [25, Data(x=[735, 34], edge_index=[2, 1502]), 14],
 [26, Data(x=[863, 34], edge_index=[2, 1677]), 20],
 [27, Data(x=[926, 34], edge_index=[2, 1797]), 18],
 [28, Data(x=[1111, 34], edge_index=[2, 2138]), 16],
 [29, Data(x=[1161, 34], edge_index=[2, 2216]), 16],
 [30, Data(x=[1637, 34], edge_index=[2, 3267]), 17],
 [31, Data(x=[3215, 34], edge_index=[2, 6624]), 18],
 [32, Data(x=[3977, 34], edge_index=[2, 7808]), 20],
 [33, Data(x=[4278, 34], edge_index=[2, 8310]), 19],
 [34, Data(x=[4618, 34], edge_index=[2, 9031]), 19],
 [35, Data(x=[4953, 34], edge_index=[2, 9545]), 19],
 [36, Data(x=[5037, 34], edge_index=[2, 10750]), 19],
 [37, Data(x=[9268, 34], edge_index=[2, 20010]), 20],
 [38, Data(x=[10336, 34], edge_index=[2, 21728]), 21],
 [39, Data(x=[10408, 34], edge_index=[2, 21818]), 24],
 [40, Data(x=[12014, 34], edge_index=[2, 24871]), 19],
 [41, Data(x=[12520, 34], edge_index=[2, 25639]), 20],
 [42, Data(x=[14191, 34], edge_index=[2, 32765]), 22]]



In [ ]:
diameters = np.array([np.array(lst) for lst in diameters])

In [ ]:
diameters.shape

In [ ]:
gSize = []
for i in range(43):
   gSize.append(gnn_re_43[i].num_nodes)
plt.stem(gSize);

In [ ]:
np.median(gSize)

In [ ]:
sns.histplot(gSize, bins=20) #[20,50,100,200,500,800,1000,5000,10000,20000]);
plt.xlabel('Number of Nodes');
plt.ylabel('Number of Graphs');
plt.title('Histogram of GNN-RE Dataset Graph Sizes');

In [ ]:
np.median(gSize)

In [ ]:
diameters[:,2]

In [ ]:
np.average(diameters[:,2].astype(int))

In [ ]:
sns.histplot(diameters[:,2].astype(int), bins = 20);
plt.xlabel('Diameter');
plt.ylabel('Number of Graphs');
plt.title('Histogram of GNN-RE Dataset Diameters');

# Generate Permutations of GNN_RE

In [ ]:
# !pip install deeprobust

In [ ]:
from deeprobust.graph.data import Dataset
from deeprobust.graph.global_attack import Random

from deeprobust.graph.data import Dataset, Dpr2Pyg, Pyg2Dpr


In [ ]:
def generate_isomorphic_permutations(G, num_samples=100): 

    def create_isomorph(G):
        num_nodes = G.num_nodes

        # Generate a permutation of the nodes
        permutation = torch.randperm(num_nodes)

        # Apply the permutation to the node features
        G.x = G.x[permutation]

        # Create a mapping tensor
        mapping_tensor = torch.tensor(permutation)

        # Apply the permutation to the edges
        G.edge_index = mapping_tensor[G.edge_index]

        return G
    dataset = []

    for _ in range(num_samples):
        # Create a deep copy of the graph
        G_copy = copy.deepcopy(G)

        # Create an isomorphic graph and add it to the dataset
        G_isomorph = create_isomorph(G_copy)
        dataset.append(G_isomorph)

    return dataset

def generate_nonisomorphic_permutations(graph,num_samples):

    model = Random()
    non_isomorphic_graphs = []
    while len(non_isomorphic_graphs) < num_samples:
        # Create a deep copy of the graph
        graph_copy = copy.deepcopy(graph)
        num_edges = graph.num_edges

        adj = pyg_to_csr(graph_copy)  # Convert to CSR matrix
        n_perturbations = random.randint(max(1, num_edges//2), num_edges-1)

        model.attack(adj, n_perturbations=n_perturbations, type='remove')  # remove random amount of edges
        modified_adj1 = model.modified_adj  # Get the modified adjacency matrix

        model.attack(modified_adj1, n_perturbations=n_perturbations, type='add')  # add (same) random amount of edges back 
        modified_adj2 = model.modified_adj  # Get the modified adjacency matrix

        tri_mod = remove_redundant_edges_csr(modified_adj2)

        non_isomorphic_graph = csr_to_pyg(tri_mod, graph_copy.x, graph_copy.y)  # Convert back to PyG

        # Convert the graphs to networkx for WL hashing
        graph_nx = to_networkx(graph, to_undirected=True) #although it says "to undirected = true", when we convert back to PYG, it becomes directed.
        non_isomorphic_graph_nx = to_networkx(non_isomorphic_graph, to_undirected=True)  #although it says "to undirected = true", when we convert back to PYG, it becomes directed.

        # Check if the WL hashes are different
        if nx.weisfeiler_lehman_graph_hash(graph_nx) != nx.weisfeiler_lehman_graph_hash(non_isomorphic_graph_nx):
            non_isomorphic_graphs.append(non_isomorphic_graph)

    return non_isomorphic_graphs

In [ ]:
def generate_permutations(G, k):
    '''G: A list of graphs you want to make permutations of
        k: the number of permutations per graph
        '''
    data_list = []
    for graph in tqdm(G):

        if graph.num_nodes < 10 or graph.num_edges == (graph.num_nodes*(graph.num_nodes-1))/2: #we can not generate non-isomorphic graphs if less than 10 nodes or fully connected graphs

            # Generate isomorphic graphs
            isomorphic_graphs = generate_isomorphic_permutations(graph, num_samples=2*k)
            # Pair each isomorphic graph with the original graph and assign label 1.0
            data_list.extend([(graph, isomorphic_graph, 1.0) for isomorphic_graph in isomorphic_graphs])
            
        else:
        
            # Generate isomorphic graphs
            isomorphic_graphs = generate_isomorphic_permutations(graph, num_samples=k)
            # Pair each isomorphic graph with the original graph and assign label 1.0
            data_list.extend([(graph, isomorphic_graph, 1.0) for isomorphic_graph in isomorphic_graphs])

            # Generate non-isomorphic graphs
            non_isomorphic_graphs = generate_nonisomorphic_permutations(graph, num_samples=k)
            # Pair each non-isomorphic graph with the original graph and assign label 0.0
            data_list.extend([(graph, non_isomorphic_graph, 0.0) for non_isomorphic_graph in non_isomorphic_graphs])
            
    return data_list

In [ ]:
k = 16 #the number of permutations.

In [ ]:
#note that the '.dataset' function of PYG graphs SHOULD NOT be used as it returns the entirety of gnnre43

In [ ]:
#For reproducability, we hardcode the train and test set
gnnRETrain = gnn_re_43[::2]  # Elements at even indices
gnnRETest = gnn_re_43[1::2]  # Elements at odd indices

In [ ]:
largeD_IDX = [11, 23,37, 39, 41] #the largest diameter graphs in the test set
largeDTestList = [gnn_re_43[g] for g in largeD_IDX]

largestN_idx = [37,39,41]
largestN_graphs_list = [gnn_re_43[g] for g in largestN_idx] #largest num nodes part of the test set 
 
print(len(gnnRETest), len(gnnRETrain), len(largeDTestList), len(largestN_graphs_list))

In [ ]:
#takes ~12 min to run
# gnnRETrain = generate_permutations(gnnRETrain, k)
# gnnRETest = generate_permutations(gnnRETest, k)
# largeDTest = generate_permutations(largeDTestList, k)
# largestN_graphs = generate_permutations(largestN_graphs_list, k) 

In [ ]:

# with gzip.open('gnnRETrain.pkl.gz', 'wb') as f:
#     pickle.dump(gnnRETrain, f)

# with gzip.open('gnnRETest.pkl.gz', 'wb') as f:
#     pickle.dump(gnnRETest, f)

# with gzip.open('largeDTest.pkl.gz', 'wb') as f:
#     pickle.dump(largeDTest, f)

# with gzip.open('largestN_graphs.pkl.gz', 'wb') as f:
#     pickle.dump(largestN_graphs, f)

In [ ]:
with gzip.open('gnnRETrain.pkl.gz', 'rb') as f:
    gnnRETrain = pickle.load(f)

with gzip.open('gnnRETest.pkl.gz', 'rb') as f:
    gnnRETest = pickle.load(f)

with gzip.open('largeDTest.pkl.gz', 'rb') as f:
    largeDTest = pickle.load(f)

with gzip.open('largestN_graphs.pkl.gz', 'rb') as f:
    largestN_graphs = pickle.load(f)

In [ ]:
largeDTestList

In [ ]:
largestN_graphs_list

# Generate Artificial Dataset

In [ ]:
def generate_synthetic_CGI(num_graphs, num_iso_pairs, num_features, min_nodes=5, max_nodes=7):
    """
    Generates a list of graph pairs and their corresponding labels for identifying circuit graph isomorphisms.
    Takes 7 minutes to make 50 graphs of size 2000 nodes

    Args:
        num_graphs (int): The total number of connected graphs to generate.
        num_iso_pairs (int): The number of isomorphic pairs to include in the data_list.
        num_features (int): The number of features for each node in the graphs.
        min_nodes (int): The minimum number of nodes in each graph.
        max_nodes (int): The maximum number of nodes in each graph.

    Returns:
        data_list (list): A list of graph pairs and their corresponding labels, where each pair consists of two
        Data objects and a label (1 for isomorphic pair, 0 for non-isomorphic pair).
    """

    
    def generate_isomorphic_pair(num_nodes):
            """
            Generates a pair of isomorphic graphs with `num_nodes` nodes.

            Args:
                num_nodes (int): The number of nodes in each graph.

            Returns:
                A tuple containing two NetworkX graphs which are isomorphic.
            """
            # Generate a random graph
            #G1 = nx.barabasi_albert_graph(num_nodes,random.randint(2, num_nodes-1)) #this produces HARD graphs for vf2. 
            G1 = nx.gnm_random_graph(num_nodes,random.randint(num_nodes, num_nodes*(num_nodes-1)//2))

            # Shuffle the nodes to generate a second graph
            nodes = list(G1.nodes())
            random.shuffle(nodes)
            mapping = {n: nodes[i] for i, n in enumerate(G1.nodes())}
            G2 = nx.relabel_nodes(G1, mapping)

            #assert(nx.is_c              onnected(G1) == True and nx.is_connected(G2) == True)

            return G1, G2

    def generate_non_isomorphic_pair(num_nodes, num_edges):
        while True:
            #G1 = nx.barabasi_albert_graph(num_nodes, num_edges, seed=123) #if you want to make things really hard for vf2
            G1 = nx.gnm_random_graph(num_nodes, num_edges, seed=random.randint(2, num_nodes-1) )
            G2 = nx.gnm_random_graph(num_nodes, num_edges, seed=random.randint(2, num_nodes-1) )
            if nx.weisfeiler_lehman_graph_hash(G1) != nx.weisfeiler_lehman_graph_hash(G2):
                return G1, G2

    data_list = []

    for i in tqdm(range(num_iso_pairs), leave = False):
        num_nodes = random.randint(min_nodes, max_nodes)
        G1, G2 = generate_isomorphic_pair(num_nodes)
        features = torch.randn((num_nodes, num_features))
        graph1 = Data(edge_index=torch.tensor(list(G1.edges)).t().contiguous(), x=features)
        graph2 = Data(edge_index=torch.tensor(list(G2.edges)).t().contiguous(), x=features)
        data_list.append((graph1, graph2, 1.0))

    assert(num_graphs - num_iso_pairs > 0)

    for i in tqdm(range(num_graphs - num_iso_pairs), leave=False):
        num_nodes = random.randint(min_nodes, max_nodes)
        num_edges = random.randint(2, num_nodes-1)
        G1, G2 = generate_non_isomorphic_pair(num_nodes, num_edges)
        features1 = torch.randn((num_nodes, num_features))
        features2 = torch.randn((num_nodes, num_features))
        graph1 = Data(edge_index=torch.tensor(list(G1.edges)).t().contiguous(), x=features1)
        graph2 = Data(edge_index=torch.tensor(list(G2.edges)).t().contiguous(), x=features2)
        data_list.append((graph1, graph2, 0.0))

    return data_list

In [ ]:
synthetic_data = generate_synthetic_CGI(800, 400, 34, min_nodes=50, max_nodes=50)

# Set Data Loaders

In [ ]:
#Synthetic Dataloaders

# Create your dataset and split it into training and test sets
synDS = DataLoader(synthetic_data)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(synthetic_data, [0.8, 0.1, 0.1])


In [ ]:
# with gzip.open('synTrain.pkl.gz', 'wb') as f:
#     pickle.dump(train_dataset, f)

# with gzip.open('synVal.pkl.gz', 'wb') as f:
#     pickle.dump(val_dataset, f)

# with gzip.open('synTest.pkl.gz', 'wb') as f:
#     pickle.dump(test_dataset, f)

In [ ]:
with gzip.open('synTrain.pkl.gz', 'rb') as f:
    syn_train_dataset = pickle.load(f)

with gzip.open('synVal.pkl.gz', 'rb') as f:
    syn_val_dataset = pickle.load(f)

with gzip.open('synTest.pkl.gz', 'rb') as f:
    syn_test_dataset = pickle.load(f)

In [ ]:

# Create your data loaders
train_loaderSyn = DataLoader(syn_train_dataset, batch_size=8, shuffle=True)
val_loaderSyn = DataLoader(syn_val_dataset, batch_size=8, shuffle=False)
test_loaderSyn = DataLoader(syn_test_dataset, batch_size=1, shuffle=False)

In [ ]:
print(len(train_loaderSyn.dataset), len(val_loaderSyn.dataset), len(test_loaderSyn))

In [ ]:
type(train_loaderSyn)

In [ ]:
# RE Dataloaders

train_datasetRE =  DataLoader(gnnRETrain, batch_size = 8, shuffle = True)
test_datasetRE = DataLoader(gnnRETest, batch_size = 1, shuffle = False)
largeD_datasetRE = DataLoader(largeDTest, batch_size=1, shuffle = False)
largestN_graphsDL = DataLoader(largestN_graphs, batch_size = 1, shuffle = False) 

In [ ]:
print(len(train_datasetRE.dataset), len(test_datasetRE.dataset), len(largeD_datasetRE))

In [ ]:
type(train_datasetRE)

# Full WL SGNN (AKA DEPTH GRAPHNET)

In [ ]:

class FullWLGraphSiameseNet(torch.nn.Module):
    def __init__(self, num_node_features=numREFeatures, hidden_dim=36, encoding_size=32, num_layers=29): #num layers set to the max diameter
        super(FullWLGraphSiameseNet, self).__init__()

        # Define MLP for GINConv
        nn1 = torch.nn.Sequential(torch.nn.Linear(num_node_features, hidden_dim),
                                  torch.nn.ReLU(),
                                  torch.nn.Linear(hidden_dim, hidden_dim))
        
        self.num_layers = num_layers

        # Define the GINConv layers for the encoder network
        self.convs = torch.nn.ModuleList()
        self.convs.append(GINConv(nn1))
        for _ in range(num_layers - 1):
            nn2 = torch.nn.Sequential(torch.nn.Linear(hidden_dim, hidden_dim),
                                      torch.nn.ReLU(),
                                      torch.nn.Linear(hidden_dim, hidden_dim))
            self.convs.append(GINConv(nn2))

        self.encoder = torch.nn.Linear(hidden_dim, encoding_size)

        # Define the layers for the classification network
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(encoding_size, encoding_size),
            torch.nn.ReLU(),
            torch.nn.Linear(encoding_size, encoding_size),
            torch.nn.ReLU(),
            torch.nn.Linear(encoding_size, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, graph1, graph2):
        x1, edge_index1 = graph1.x, graph1.edge_index
        x2, edge_index2 = graph2.x, graph2.edge_index

        for i in range(self.num_layers):
            x1 = self.convs[i](x1, edge_index1)
            x2 = self.convs[i](x2, edge_index2)
            x1 = F.relu(x1)
            x2 = F.relu(x2)

        # Encode the two graphs
        encoding1 = self.encoder(x1)
        encoding2 = self.encoder(x2)

        # Apply add/mean pooling to obtain fixed-size representations
        encoding1 = global_add_pool(encoding1, graph1.batch) #global_mean_pool(encoding1, graph1.batch)
        encoding2 = global_add_pool(encoding2, graph2.batch)#global_mean_pool(encoding2, graph2.batch)

        # Compute the difference between the encodings
        diff = torch.abs(encoding1 - encoding2)

        # Pass the difference through the classification network
        out = self.classifier(diff.view(-1, diff.shape[1]))

        return out.squeeze()

# Train 29 Layer

In [ ]:
l = 29
model_RE_Full = FullWLGraphSiameseNet(num_node_features=34,hidden_dim=36, encoding_size=32, num_layers=l)
model_RE_Full = model_RE_Full.to(device)
num_epochs = 30
# Define the loss function and optimizer
criterion = nn.BCELoss() #nn.MSELoss() #experiment with mse, nn.BCELoss()
optimizer = torch.optim.Adam(model_RE_Full.parameters(), lr=1e-4)

# Count the number of parameters
total_params = sum(p.numel() for p in model_RE_Full.parameters())
print(f"Total number of parameters: {total_params}")
# num_params.append(total_params)

# Train the model on your dataset. FULLWLSGNN takes a 5 min for 30 epochs
for epoch in range(num_epochs):
    for i, (g1, g2, label) in enumerate(train_datasetRE):
        # print(g1, g2, label)
        # assert(False)
        g1 = g1.to(device)
        g2 = g2.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model_RE_Full(g1, g2)
        # print(label.dtype, output.dtype)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 2 == 0 and (i+1) % 35 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_datasetRE), loss.item()))

#num params 80513

In [ ]:
# Test the model on your test dataset
correct = 0
total = 0
with torch.no_grad():
    
    for data1, data2, label in test_datasetRE:
        data1 = data1.to(device)
        data2 = data2.to(device)
        label = label.to(device)
        output = model_RE_Full(data1, data2)
        # print('Num Nodes', data1.num_nodes)
        # print(output.shape[0])
        # print(label.shape)
        # _, predicted = torch.max(output, 1)
        predicted = torch.round(output)
        total += label.size(0)
        correct += (predicted == label).sum().item()
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%', )#' using ', l, ' message passing layers')

In [ ]:
# Train 29 Layer GREED
#Greed needs a slightly different dataloader than the others. (the y value must be duplicated) 
class GREEDDataLoader:
    def __init__(self, original_dataloader):
        self.original_dataloader = original_dataloader

    def __iter__(self):
        for g1, g2, y in self.original_dataloader:
            yield g1, g2, y, y

    def __len__(self):
        return len(self.original_dataloader)

In [ ]:

from neuro import models, config, train
greed_train_loader = GREEDDataLoader(train_datasetRE)
greed_val_loader = GREEDDataLoader(test_datasetRE)
config.device# = torch.device('cpu')


In [ ]:
#I am using this for timing so i don't actually need to train this. epoch set to just 1
greed = models.NeuralSiameseModel(n_layers= 20, input_dim=34, hidden_dim=36, output_dim= 32)
ad = torch.optim.Adam(greed.parameters(), lr=2e-4)
print(total_params := sum(p.numel() for p in greed.parameters()))
train.train_loop(model = greed, opt=ad, 
                        loader=greed_train_loader, n_epochs=1)

#num params: 120729